In [1]:
import os
import pytesseract
from docx2pdf import convert
import subprocess
import time
import win32com.client  # Убедитесь, что этот импорт присутствует
import fitz  # PyMuPDF
from PIL import Image, ImageDraw, ImageFont

import tkinter as tk
from tkinter import messagebox
from docx import Document

In [2]:
#  ----   Основные переменные для пользоваетля

input_dir = './input' # в этой папке лежат .docx-файлы лекций
pdf_dir = './pdf' # в эту папку скрипт сконвертирует .docx-файлы из input_dir. (это необходимо, т.к. скрипт видит страницы именно в pdf формате (а не в docx))

 # в этой папке скрипт создаст (для каждой лекции из input) свою подпапку
output_dir = './output'
# в папке output, в подпапке каждой лекции лежат две папки: to_combine и to_print

# В этой папке лежат сделанные из pdf картинки страниц текста
# эта папка лежит в output_dir.
to_combine_dir = './to_combine' 

# В этой папке лежат объединенные специфическим образом для печати на А4 картинки из папки to_combine
# Это и есть готовые конспекты к печати
to_print = './to_print'

# Загружаем шрифт нашего почерка@
font = ImageFont.truetype("./fonts/MyFont1.otf", size=67)

note_type = 'cells' #клеточки. note_type='lines' - лист в линейку

font_color = 'blue' #цвет шрифта

# Задаем вертикальный шаг между строками
line_spacing = 110  # 55 - писать на каждой строке, 110 - пропускать одну строку между двумя строками текста

#Адрес файла запуска Microsoft Word
word_path = r"C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE" 


In [3]:
print(font.size)

67


In [4]:
#очищаем папку output от картинок и pdf-файлов из предыдущих итераций
def delete_files_in_folder(folder):
    if os.path.exists(folder):  # Проверяем, что папка существует
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_path = os.path.join(root, file)
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.pdf')):
                    print(f'Удаление файла: {file_path}')
                    os.remove(file_path)  # Удаляем файл
    else:
        print(f"Папка {folder} не существует.")

# пользователю задаётся вопрос: очистить папки: input_dir, pdf_dir, output_dir, to_combine_dir
# пользователь может поставить галочки в чекбоксы напротив нужных ему папок
# после чего идет цикл по выбранным папкам, и функция delete_files_in_folder очищает содержимое

In [5]:
# Функция для обработки выбранных папок
def delete_selected_folders():
    selected_folders = []
    
        # Функция для поиска и удаления файлов во всех папках to_print и to_combine
    def delete_in_all_subfolders(base_folder, target_folder_name):
        for root, dirs, files in os.walk(base_folder):
            if target_folder_name in dirs:
                target_folder_path = os.path.join(root, target_folder_name)
                print(f"Очистка папки: {target_folder_path}")
                delete_files_in_folder(target_folder_path)
    
    # Проверяем, какие чекбоксы выбраны
    if var_pdf_dir.get():
        selected_folders.append(pdf_dir)
    if var_to_print.get():
        # Удаление файлов во всех папках to_print в папке output
        delete_in_all_subfolders('./output', 'to_print')
    if var_to_combine_dir.get():
        # Удаление файлов во всех папках to_combine в папке output
        delete_in_all_subfolders('./output', 'to_combine')

    if selected_folders:
        # Подтверждение действия для папок input и pdf
        result = messagebox.askquestion("Подтверждение", "Вы уверены, что хотите очистить выбранные папки?")
        if result == 'yes':
            for folder in selected_folders:
                print(f"Очистка папки: {folder}")
                delete_files_in_folder(folder)  # Очищаем выбранную папку
            messagebox.showinfo("Готово", "Выбранные папки успешно очищены.")
        else:
            messagebox.showinfo("Отмена", "Очистка папок отменена.")

In [6]:
#В папке output могут быть сколько угодно разных папок, внутри которых могут быть папки to_print и to_combine_dir
#Если пользователь выберет чекбокс удалить to_print/to_combine_dir, то удалится должны все файлы во всех папках to_print/to_combine_dir

# Папки для очистки
input_dir = './input'
pdf_dir = './pdf'
to_print_dir = './to_print'
to_combine_dir = './to_combine'


# Создаем окно
root = tk.Tk()
root.title("Очистка папок")

# Создаем переменные для чекбоксов
var_pdf_dir = tk.BooleanVar()
var_to_combine_dir = tk.BooleanVar()
var_to_print = tk.BooleanVar()

# Создаем чекбоксы
tk.Checkbutton(root, text="Очистить папку pdf", variable=var_pdf_dir).pack(anchor=tk.W)
tk.Checkbutton(root, text="Очистить папку output/to_print", variable=var_to_print).pack(anchor=tk.W)
tk.Checkbutton(root, text="Очистить папку output/to_combine", variable=var_to_combine_dir).pack(anchor=tk.W)

# Кнопка для выполнения очистки
tk.Button(root, text="Очистить выбранные папки", command=delete_selected_folders).pack(pady=10)

# Запуск окна
root.mainloop()


In [7]:
def create_dir_if_it_doesnt_exist(root_dir, dir):
    path = os.path.join(root_dir, dir,)
        
    if not os.path.exists(path):
        os.makedirs(path)
        
create_dir_if_it_doesnt_exist('./', input_dir)
create_dir_if_it_doesnt_exist('./', pdf_dir)
create_dir_if_it_doesnt_exist('./', output_dir)


In [8]:
img_half_empty = './img/empty_half.png'

In [9]:
# Убедитесь, что путь к изображению корректный и изображение загружено
img_template_path=''

if note_type == 'cells':
    img_template_path = './img/cells_half.png'
else:
    img_template_path = './img/lines_half.png'


if not os.path.exists(img_template_path):
    raise FileNotFoundError(f"Изображение не найдено по пути: {img_template_path}")

# Открываем изображение с клетками img_half_empty
#img_template = Image.open(img_half_empty)

img_template = Image.open(img_template_path)

In [10]:
def run_ms_word(word_path):
    # Запускаем Microsoft Word
    try:
        is_ms_word_closed = subprocess.Popen([word_path])
        print("Microsoft Word запущен.")
    except Exception as e:
        print(f"Не удалось запустить Microsoft Word: {e}")
        

In [11]:
def is_ms_word_running(process_name="WINWORD.EXE"):
    """Проверяет, работает ли Microsoft Word и полностью ли он загружен."""
    try:
        # Получаем список процессов и проверяем наличие Microsoft Word
        output = subprocess.check_output(["tasklist"], text=True)
        
        if process_name in output:
            print("MS Word работает.")
            
            # Дожидаемся полной загрузки Word
            while True:
                try:
                    # Проверяем, доступен ли объект Word
                    word_app = win32com.client.GetActiveObject("Word.Application")
                    if word_app is not None:
                        print("MS Word полностью загружен и готов к работе.")
                        return True
                except Exception:
                    # Если объект недоступен, продолжаем ожидание
                    print("Ожидание полной загрузки MS Word...")
                    time.sleep(1)  # Ждем 1 секунду перед следующей проверкой
            
        else:
            print("MS Word не запущен.")
            return False
    except Exception as e:
        print(f"Ошибка при проверке процессов: {e}")
        return False
    

In [12]:
for filename in os.listdir(input_dir):
    if ".docx" in filename.lower() and "$" not in filename:
        print(filename)
        
        filename_no_ext = filename[:-5]
        temp_pdf_path = (f"{pdf_dir}/{filename_no_ext}.pdf")
        

        # Если Word запущен через subroccess:
        if is_ms_word_running():
             # Microsoft Word не успевает запуститься до запуска этой строки. Как исправить?
            convert( (f"{input_dir}/{filename}"), temp_pdf_path)
        
        
        #Иначе запускаем его и 
        else:
            run_ms_word(word_path)
            # Как здесь дождаться когда Microsoft запустится? После этого нужно выполнить этот код:
            

            # Ожидаем, пока Microsoft Word не запустится
            while not is_ms_word_running():
                print("Ожидание запуска Microsoft Word...")
                time.sleep(2)  # Ждем 1 секунду перед следующей проверкой

            # После того как Microsoft Word запущен, выполняем конвертацию
            try:
                convert(os.path.join(input_dir, filename), temp_pdf_path)
                print(f'Конвертировано: {input_dir}/{filename} в {temp_pdf_path}')
            except Exception as e:
                print(f'Ошибка при конвертации: {e}')

  
        print(f"{input_dir}/{filename}")
        

Лабораторная_работа_1.4.docx
MS Word не запущен.
Microsoft Word запущен.
MS Word работает.
Ожидание полной загрузки MS Word...
MS Word полностью загружен и готов к работе.


  0%|          | 0/1 [00:00<?, ?it/s]

Конвертировано: ./input/Лабораторная_работа_1.4.docx в ./pdf/Лабораторная_работа_1.4.pdf
./input/Лабораторная_работа_1.4.docx


In [13]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)
        
    if os.path.isfile(input_path) and 'docx' in input_filename.lower() and '$' not in input_filename:
        print(input_filename)
        create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5])

Лабораторная_работа_1.4.docx


In [14]:

# Функция для обработки текста с форматированием из docx
def extract_text_with_styles(docx_path):
    doc = Document(docx_path)
    styled_text = []
    
    # Перебираем все параграфы
    for paragraph in doc.paragraphs:
        paragraph_styles = {
            'alignment': paragraph.alignment  # Горизонтальное выравнивание (0 - слева, 1 - центр, 2 - справа)
        }
        line = []
        for run in paragraph.runs:
            text = run.text.strip()
            styles = []
            
            if run.bold:
                styles.append('bold')
            if run.italic:
                styles.append('italic')
            if run.underline:
                styles.append('underline')
            if run.font.superscript:
                styles.append('superscript')
            if run.font.subscript:
                styles.append('subscript')
            
            line.append({
                'text': text,
                'styles': styles
            })
        
        if line:
            styled_text.append({
                'line': line,
                'paragraph_styles': paragraph_styles
            })
    
    return styled_text

In [15]:
for input_filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, input_filename)

    if os.path.isfile(input_path) and 'docx' in input_filename.lower() and '$' not in input_filename:
        #print(input_filename)
        create_dir_if_it_doesnt_exist(output_dir, input_filename[:-5])

        # Открываем PDF-документ
        pdf_path = os.path.join(pdf_dir, f'{input_filename[:-5]}.pdf')
        
        create_dir_if_it_doesnt_exist(f'{output_dir}/{input_filename[:-5]}', to_combine_dir)

        doc = fitz.open(pdf_path)

        # Перебираем страницы PDF
        for page_num in range(len(doc)):
            # Получаем страницу
            page = doc.load_page(page_num)
            
            # Извлекаем текст с форматированием
            text = page.get_text("text")  # Извлекаем текст с форматированием
        
            # Разбиваем текст на отдельные строки
            text_lines = text.splitlines()
            
            # Создаем копию шаблона для каждой страницы
            #img = img_template.copy()
            #draw = ImageDraw.Draw(img)
            
            
            
            
             
            # Берем высоту буквы по англ. символу "a"
            char_a = 'a' 
            line_height = font.getsize(char_a)[1]  # Получаем высоту строки
                
            canvas_image_path = './img/cells_half.png'
            line_image_path = './img/line_empty.png'    
            
            # Переносим строки текста на соответствующие линии
            for i, line in enumerate(text_lines):
                
                
                                
                y_position = (i + 2) * line_spacing - line_height  # Выравниваем низ текста по линии
                
                coeff = 20
                
                canvas_image = Image.open(canvas_image_path)
                
                line_img = Image.open(line_image_path)                
                draw = ImageDraw.Draw(line_img)
                
                print(f'Рисуем текст на странице {page_num + 1}, строка {i + 1}: {line}')
                #draw.text((4 * 55, y_position), line, fill=font_color, font=font)
                draw.text((0,0), line, fill=font_color, font=font)
                
                resized_line_img = line_img.resize((1660, line_spacing*2-coeff))
                
                
                line_img.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/line {page_num + 1}.png')  # Нумерация страниц начинается с 1
                
                #resized_line_img.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/resized_line {page_num + 1}.png')  # Нумерация страниц начинается с 1


                #вставить в канвас картинку(строка, координаты, маска картинки)
                canvas_image.paste(resized_line_img, (4*55, y_position+int(coeff/2)), resized_line_img)

                
            # Сохраняем результат для текущей страницы
                canvas_image.save(f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/{page_num + 1}.png')  # Нумерация страниц начинается с 1
                canvas_image_path=f'{output_dir}/{input_filename[:-5]}/{to_combine_dir}/{page_num + 1}.png'

        # Закрываем PDF-документ
        doc.close()

Рисуем текст на странице 1, строка 1:                      ЛAабораторная работа 1. 4 


C:\Users\Personal Computer\AppData\Local\Temp\ipykernel_34392\1611611034.py:36: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize(char_a)[1]  # Получаем высоту строки


TypeError: 'float' object cannot be interpreted as an integer